<a href="https://colab.research.google.com/github/jbpost2/ST-554-Big-Data-With-Python-Course-Notes/blob/main/02_SQL_Map_Reduce_pyspark/mapreduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Map Reduce Example

You'll need to place the `charles-dickens-oliver-twist.txt` file into your colab session.

In [ ]:
import string
#read in the book as a string
with open('charles-dickens-oliver-twist.txt', 'r') as f:
    my_lines = f.read()

#process and write out each chapter to its own txt file
def find_chap(lines, string):
    chap_start = lines.find(string)
    chap_end = lines.find(string, chap_start + 1)
    if chap_end == -1:
        chap_end = lines.find("End of the Project Gutenberg EBook")
    return([chap_start, chap_end])

def remove_char(lines):
    #replace punctuation
    for symbol in string.punctuation:
        lines = lines.replace(symbol, "")
    lines = lines.replace("\n", " ")
    return(lines)

def save_chap(lines, chap = None):
    if chap == None:
        start_end = find_chap(lines, "CHAPTER I")
        chap = 1
    else:
        start_end = find_chap(lines, "CHAPTER")
    #get the chapter and turn it to lower case
    chap_text = lines[start_end[0]:start_end[1]].lower()
    #remove punctuation
    chap_text = remove_char(chap_text)
    with open('chap' + str(chap) + '.txt', 'w') as w:
        w.write(chap_text)
    chap += 1
    if lines[(start_end[1] + 1):].find("CHAPTER") == -1:
        return
    else:
        save_chap(lines[start_end[1]:], chap = chap)

In [ ]:
save_chap(my_lines)

In [ ]:
def map_words(chap):
    word_count_dictionary = {}
    chap_split = chap.split(" ")
    for word in chap_split:
        if word in word_count_dictionary:
            word_count_dictionary[word] += 1
        else:
            word_count_dictionary[word] = 1
    return word_count_dictionary

In [ ]:
with open('chap1.txt', 'r') as f:
    my_chap = f.read()
chap1dict = map_words(my_chap)

with open('chap2.txt', 'r') as f:
    my_chap = f.read()
chap2dict = map_words(my_chap)

with open('chap3.txt', 'r') as f:
    my_chap = f.read()
chap3dict = map_words(my_chap)

def word_reduce(dict1, dict2):
    combined = {}
    for key in dict1.keys():
        if key in dict2:
            combined[key] = dict1[key] + dict2[key]
        else:
            combined[key] = dict1[key]
    for key in dict2.keys():
        if key not in dict1.keys():
            combined[key] = dict2[key]
    return combined


In [ ]:
word_reduce(word_reduce(chap1dict, chap2dict), chap3dict)

chapters = []
for i in range(1, 54):
    with open('chap' + str(i) + '.txt', 'r') as f:
        my_chap = f.read()
    chapters.append(map_words(my_chap))

import functools
final = functools.reduce(word_reduce, chapters)

In [ ]:
final["of"]

3686